In [1]:
import tools.processing as pre
import re

# use less text for now to avoid memory error
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

TIMESTEPS = 6

vocab = pre.Vocabulary(text)

# double \\n to avoid null error in tensorboard projection
text = text.replace("\n", " \\n ")

# remove extra spacing
tokens = re.sub( " +", " ", text).split(" ")[:-1]


str_data, str_labels = pre.create_data_label_pairs(tokens, TIMESTEPS)

print( list( zip(str_data, str_labels) )[:5] )

[(['as', 'real', 'as', 'it', 'seems', 'the'], 'american'), (['real', 'as', 'it', 'seems', 'the', 'american'], 'dream'), (['as', 'it', 'seems', 'the', 'american', 'dream'], '\\n'), (['it', 'seems', 'the', 'american', 'dream', '\\n'], "ain't"), (['seems', 'the', 'american', 'dream', '\\n', "ain't"], 'nothing')]


In [2]:
import tools.training as tr

encoder = tr.IndexWordEncoder("Index-Word-Encoding", vocab.word2index)
decoder = tr.OneHotWordDecoder("1-Hot-Word-Decoding", vocab.index2word, temperature=0.8)

data = encoder.encode( str_data )
labels = encoder.encode_labels( str_labels )

my_seed = " ".join(str_data[0]).replace("\\n", "\n")

del str_data, str_labels

In [3]:
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = len(vocab.word2index)

EPOCHS = 12
BATCH_SIZE = 256

EMBEDDING_SIZE = 256

rnn = nn.EmbeddedSingleLayerRNN(name = "multi-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, EMBEDDING_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, _: tr.sample( my_seed, trainable, encoder, decoder, length=50)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE)

Building model from scratch! 
 Saving into: 'logs/train_model'


Epoch 1/12
Loss:    	 5.617822170257568
Accuracy:	 13.97268295288086
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the 'tack 
 when my the silent 
 thinking 
 nine off crew contact of mysterious lot girlfriend one where's the 
 a krsone just and me 
 bolo 
 got luck motherfuckin' love cops words 
 
 what the fakes 
 got up comes back 
 this what just didn't


Epoch 2/12
Loss:    	 5.106150150299072
Accuracy:	 17.80001449584961
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the gun scared 
 thoughts outta force we beat style the other she's and my sayin 
 now that i'm you tha leave how that back 
 before pack man day closer 
 you family get baby 
 me it the goals pal this 
 when i rate the pal that


Epoch 3/12
Loss:    	 4.360679626464844
Accuracy:	 23.811262130737305
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real 

In [3]:
decoder.temperature = 0.5

sampler = lambda seed_text: tr.sample( seed_text, rnn, encoder, decoder, length=40)

sampler("as real as it seems the")

NameError: name 'rnn' is not defined

In [7]:
decoder = tr.MaxOneHotDecoder("1-Hot-Word-Decoding", vocab.index2word)

sampler = lambda seed_text: tr.sample( seed_text, rnn, encoder, decoder, length=40)

sampler("as real as it seems the")

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = len(vocab.word2index)

TIMESTEPS = 6
EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 256

rnn = nn.EmbeddedSingleLayerRNN(name = "multi-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, EMBEDDING_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, _: tr.sample( my_seed, trainable, encoder, decoder, length=50)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE)